# BRASIL EM DADOS | INTRODUÇÃO À CIÊNCIA DOS DADOS

**Grupo:** Estela Miranda Batista, 3305 | João Marcos Alves Modesto Ramos, 3506 | Roniel Barbosa, 

In [2]:
# Importando as Bibliotecas Utilizadas
import pandas as pd

# Dataframe de Covid
path1 = "./Dados/Covid/HIST_PAINEL_COVIDBR_2020_Parte1_22ago2021.csv"
path2 = "./Dados/Covid/HIST_PAINEL_COVIDBR_2020_Parte2_22ago2021.csv"
path3= "./Dados/Covid/HIST_PAINEL_COVIDBR_2021_Parte1_22ago2021.csv"
path4 = "./Dados/Covid/HIST_PAINEL_COVIDBR_2021_Parte2_22ago2021.csv"
path = [path1,path2,path3,path4]
df_from_each_file = (pd.read_csv(f,error_bad_lines=False,sep=';') for f in path)
dadosCovid = pd.concat(df_from_each_file, ignore_index=True)

# Dataframe de Vacina
path5 = "./Dados/Vacina/dadosTipoVacina.xlsx"
dadosTipoVacina = pd.read_excel(path5)

# Dados Eleições
votosEleicao = {
    'Acre': 62.24,
    'Alagoas': 34.40,
    'Amapá': 40.74,
    'Amazonas': 43.48,
    'Bahia': 23.41,
    'Ceará': 21.74,
    'Distrito Federal': 58.37,
    'Espírito Santo': 54.76,
    'Goiás': 57.24,
    'Maranhão': 24.28,
    'Mato Grosso': 60.04,
    'Mato Grosso do Sul': 55.06,
    'Minas Gerais': 48.31,
    'Paraná': 56.89,
    'Paraíba': 31.30,
    'Pará': 36.19,
    'Pernambuco': 30.57,
    'Piauí': 18.76,
    'Rio de Janeiro': 59.79,
    'Rio Grande do Norte': 30.21,
    'Rio Grande do Sul': 52.63,
    'Rondônia': 62.24,
    'Roraima': 62.97,
    'Santa Catarina': 65.82,
    'Sergipe': 27.21,
    'São Paulo': 53.00,
    'Tocantins': 44.64,
}

dadosEleicao = pd.DataFrame(votosEleicao.items(), index=votosEleicao.keys())

# Dados PIB
PIB = {
    'Acre': 15331,
    'Alagoas': 54413,
    'Amapá': 16795,
    'Amazonas': 100109,
    'Bahia': 286240,
    'Ceará': 155904,
    'Distrito Federal': 254817,
    'Espírito Santo': 137020,
    'Goiás': 195682,
    'Maranhão': 98179,
    'Mato Grosso': 137443,
    'Mato Grosso do Sul': 106969,
    'Minas Gerais': 614876,
    'Paraná': 440029,
    'Paraíba': 64374,
    'Pará': 161350,
    'Pernambuco': 186352,
    'Piauí': 50378,
    'Rio de Janeiro': 758859,
    'Rio Grande do Norte': 66970,
    'Rio Grande do Sul': 457294,
    'Rondônia': 44914,
    'Roraima': 13370,
    'Santa Catarina': 298227,
    'Sergipe': 42018,
    'São Paulo': 2210562,
    'Tocantins': 35666,
}

#dadosPIB = 


/home/estela/.local/lib/python3.8/site-packages/pandas/core/reshape/concat.py:348: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  objs = list(objs)


## 🔵 PREPARAÇÃO DOS DADOS

### Dados do Covid-19

### Dados da Vacinação

### Dados das Eleições e PIB

In [ ]:
# Preparação dos Dados da Covid-19

# Remoção dos Atributos não uUilizados na Análise
dadosCovid = dadosCovid.drop(columns=['coduf','codmun','codRegiaoSaude','nomeRegiaoSaude','Recuperadosnovos','emAcompanhamentoNovos','interior/metropolitana'])

In [ ]:
# Preparação dos Dados da Vacinação
dadosTipoVacina = dadosTipoVacina.drop(columns=['Cód. IBGE','Dose 1','Dose 2'])

In [ ]:
# Preparação dos Dados das Eleições

In [ ]:
# Preparação dos Dados do PIB

## 🟣 QUESTIONAMENTOS LEVANTADOS, E SUAS RESPOSTAS

### 🔢 Casos de Covid-19

1. Amostragem dos casos de covid por estado.
2. Qual estado tem o maior número de casos de covid?
3. Qual estado tem a maior taxa de mortos por covid?
4. Qual estado tem a maior taxa de recuperação de covid?
5. Qual a maior taxa de cresccimento de casos de covid em cada estado? Em que período isso ocorreu?
6. Qual a maior taxa de cresccimento de óbitos de covid em cada estado? Em que período isso ocorreu?
7. Qual o estado que teve a maior taxa de mortalidade? (a taxa de mortalidade consiste no número de óbitos dividido pelo número de casos)

### 📈 Taxa de Vacinação

8. Amostragem dos dados de vacinação por estado.
9. Qual o estado com maior taxa de vacinação?
10. Qual a região com maior taxa de vacinação?
11. Existe uma relação entre a taxa de vacinação e a diminuição de casos em cada estado?
12. Existe uma relação entre a taxa de vacinação e a diminuição de casos em cada região?

### 💰 Situação Socioecônomica

13. Amostragem de situação socioeconômica por estado, através do PIB.
14. Os estados com menor PIB, tiveram alguma relação com o aumento de casos de covid?
15. Os estados com menor PIB, tiveram alguma relação com o número de recuperados de casos de covid?
16. Os estados com menor PIB, tiveram alguma relação com a taxa de vacinação de casos de covid?

### 💼 Situação Política

17. Amostragem de votos durante a última eleição a favor do atual presidente.
18. O apoio ao presidente tem alguma relação com as regiões com maior taxa de casos de covid?
19. O apoio ao presidente tem alguma relação com as regiões com maior taxa de óbitos de covid?
20. O apoio ao presidente tem alguma relação com as regiões com maior taxa de vacinação?